In [1]:
from utils import *
from pathlib import Path

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
dir_path = "papers"
arxiv_ids = [2205.14135, 1904.07094, 2203.00759, 2104.08691, 2212.01681, 2210.03162, 2206.07682, 2306.08055] 

In [13]:
#text_file_path, abstract_file_path = create_corpus(dir_path, arxiv_ids)
text_file_path, abstract_file_path ='data/papers_text.txt', 'data/papers_abstract.txt'

In [17]:
dataset = create_dataset(text_file_path, abstract_file_path)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'abstract'],
        num_rows: 6
    })
    val: Dataset({
        features: ['text', 'abstract'],
        num_rows: 2
    })
})

In [18]:
tokenized_dataset = tokenize(dataset)
create_sharded_dataset(tokenized_dataset, "data", total_batches =2)

writing data/val.bin: 100%|██████████| 2/2 [00:00<00:00, 700.86it/s]


In [ ]:
remove references, use parts of the article to predict other parts, e.g. predict the abstract and conclusion

or

In [23]:
from pathlib import Path

from py_pdf_parser.loaders import load_file


import pdfminer
from pdfminer.high_level import extract_text
#%load_ext autoreload
#%autoreload 2

#import sys
#print(sys.path)

In [69]:
import os
from urllib.request import urlretrieve
import bibtexparser

dir_path = "papers"

arxiv_id = "2210.03162"
arxiv_id_bib = "2210-03162"
pdf_url = f"https://export.arxiv.org/pdf/{arxiv_id}"
bibtex_url = f"https://dblp.uni-trier.de/rec/journals/corr/abs-{arxiv_id_bib}.bib?param=1"

pdf_local_path = os.path.join(dir_path, "test.pdf")
bibtex_local_path = os.path.join(dir_path, "test.bib")

if pdf_url is not None:
    pdf_path, headers = urlretrieve(pdf_url, pdf_local_path)

if bibtex_url is not None:
    bib_path, headers = urlretrieve(bibtex_url, bibtex_local_path)


In [ ]:
def download_bibtex(arxiv_id, dir_path):

    arxiv_id_bib = str(arxiv_id).replace('.', '-')
    bibtex_url = f"https://dblp.uni-trier.de/rec/journals/corr/abs-{arxiv_id_bib}.bib?param=1"

    bibtex_local_path = os.path.join(dir_path, f"{arxiv_id_bib}.bib")

    if bibtex_url is not None:
        bib_path, _ = urlretrieve(bibtex_url, bibtex_local_path)

    return bib_path

In [121]:

def download_paper(arxiv_id, dir_path):

    pdf_url = f"https://export.arxiv.org/pdf/{arxiv_id}"

    #arxiv_id =  str(arxiv_id).replace('.', '-')
    pdf_local_path = os.path.join(dir_path, f"{arxiv_id}.pdf")

    if pdf_url is not None:
        pdf_path, _ = urlretrieve(pdf_url, pdf_local_path)

    return pdf_path



In [122]:
download_paper(2210.03162, dir_path)

'papers/2210.03162.pdf'

In [117]:
import re

def extract_metadata(bib_path):
    library = bibtexparser.parse_file(bib_path) 

    authors = library.entries[0].fields_dict['author']._value.splitlines()
    authors = [re.sub("\s{2,}"," ",elem.replace('and ','')) for elem in authors] 

    title = library.entries[0].fields_dict['title']._value
    title = re.sub("\s{2,}"," ",title)
    return title, authors

extract_metadata(bib_path)

('Prompt Compression and Contrastive Conditioning for Controllability and Toxicity Reduction in Language Models',
 ['David Wingate and', ' Mohammad Shoeybi and', ' Taylor Sorensen'])

In [99]:
title.replace('\n','')

'Prompt Compression and Contrastive Conditioning for Controllability                  and Toxicity Reduction in Language Models'

In [ ]:
from urllib.request import urlretrieve

path, headers = urlretrieve(url, filename)
for name, value in headers.items():
    print(name, value)

In [25]:
dir_path =  "papers"

def list_pdfs(dir_path):
     pathlist = Path(dir_path).glob('**/*.pdf')
     titles = []
     for path in pathlist:
          titles.append(str(path))

     return titles

def extract_text(file_path):
     return extract_text(file_path)


In [26]:
list_pdfs(dir_path)

['papers/Emergent_Abilities_of_Large_Language_Models.pdf',
 'papers/Prompt_Programming_for_Large_Language_Models_Beyond_the_Few-Shot_Paradigm.pdf']

In [13]:
text = extract_text('papers/Emergent_Abilities_of_Large_Language_Models.pdf')


In [51]:
print(text[40:350].splitlines())


['', '8', '6', '7', '0', '.', '6', '0', '2', '2', ':', 'v', 'i', 'X', 'r', 'a', '', 'Published in Transactions on Machine Learning Research (08/2022)', '', 'Emergent Abilities of Large Language Models', '', 'Jason Wei 1', 'Yi Tay 1', 'Rishi Bommasani 2', 'Colin Raﬀel 3', 'Barret Zoph 1', 'Sebastian Borgeaud 4', 'Dani Yogatama 4', 'Maarten Bosma 1', 'Denny Zhou 1', 'Donald Metzler 1', 'Ed H. Chi 1', 'Tatsu']


In [32]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open('papers/Emergent_Abilities_of_Large_Language_Models.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue()[40:80] == text[40:80])

True


In [3]:
list_pdfs(dir_path)

['papers/Emergent_Abilities_of_Large_Language_Models.pdf',
 'papers/Prompt_Programming_for_Large_Language_Models_Beyond_the_Few-Shot_Paradigm.pdf']

In [4]:
document =  load_pdf('papers/Emergent_Abilities_of_Large_Language_Models.pdf')

In [11]:
document

In [9]:
abstract = document.elements.filter_by_text_equal("2").extract_single_element()
abstract

<PDFElement tags: set(), font: 'YBSOMS+LMRoman10-Regular,10.0'>

In [10]:
document.elements.to_the_right_of(abstract).extract_single_element().text()

NoElementFoundError: There are no elements in the ElementList